In [1]:
import pandas as pd
from pathlib import Path
import logging
from box import Box
from datetime import datetime
import sys
import torch

In [ ]:
#!pip install python-box

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install /content/drive/MyDrive/

Processing ./drive/MyDrive
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for fast-bert: filename=fast_bert-2.0.3-py3-none-any.whl size=118009 sha256=2a285abfba2d14a7b2b08433bc3de7e75b62a4f267352f04dae31e0678d4284c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ibn6jhp5/wheels/b5/c7/30/2496a4be9da3477c67c27d474549190804d7788e81bd4d814b
Successfully built fast-bert
  Attempting uninstall: fast-bert
    Found existing installation: fast-bert 2.0.3
    Uninstalling fast-bert-2.0.3:
      Successfully uninstalled fast-bert-2.0.3


In [3]:
from fast_bert import BertLearner, BertDataBunch, accuracy

In [4]:
pd.set_option('display.max_colwidth', -1)
run_start_time = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [5]:
root_path = '/content/drive/My Drive'
import os

In [6]:
PATH = Path(os.path.join(root_path, "sample_data/imdb_movie_reviews"))
DATA_PATH = PATH/'data'
LABEL_PATH = PATH/'label'
OUT_PATH = PATH/'.output'
OUT_PATH.mkdir(exist_ok=True)

MODEL_PATH=OUT_PATH/'model'
MODEL_PATH.mkdir(exist_ok=True)

LOG_PATH = OUT_PATH/'logs/'
LOG_PATH.mkdir(exist_ok=True)

In [7]:
args = Box({
    "run_text": "ibdm_reviews",
    "max_seq_length": 512,
    "batch_size": 8,
    "learning_rate": 5e-3,
    "num_train_epochs": 8,
    "fp16": False,
    "model_name": 'distilroberta-base',
    "model_type": 'roberta'
})

device = torch.device('cuda') if torch.cuda.device_count() else torch.device('cpu')
if torch.cuda.device_count() > 1:
    args.multi_gpu = True
else:
    args.multi_gpu = False

In [8]:
logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

In [9]:
device

device(type='cuda')

In [10]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH, args.model_name, 
                          train_file="train_sample.csv", val_file="val_sample.csv",
                          batch_size_per_gpu=args.batch_size, 
                          max_seq_length=args.max_seq_length, 
                          multi_gpu=args.multi_gpu,
                          multi_label=False,
                          model_type=args.model_type
                         )

05/23/2022 08:20:45 - INFO - root -   Loading features from cached file /content/drive/My Drive/sample_data/imdb_movie_reviews/data/cache/cached_roberta_train_multi_class_512_train_sample.csv
05/23/2022 08:20:45 - INFO - root -   Loading features from cached file /content/drive/My Drive/sample_data/imdb_movie_reviews/data/cache/cached_roberta_dev_multi_class_512_val_sample.csv


In [11]:
metrics = [{"name": "accuracy", "function": accuracy}]

In [12]:
learner = BertLearner.from_pretrained_model(databunch, args.model_name, metrics=metrics, 
                                            device=device, multi_gpu=args.multi_gpu, is_fp16=args.fp16,
                                            multi_label=False, logging_steps=0,
                                            output_dir=OUT_PATH, logger=logger
                                           )

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias

In [13]:
learner.validate()

05/23/2022 08:20:53 - INFO - root -   Running evaluation
05/23/2022 08:20:53 - INFO - root -     Num examples = 50
05/23/2022 08:20:53 - INFO - root -     Batch size = 16


{'accuracy': 0.64, 'loss': 0.6792075335979462}

In [14]:
learner.fit(8, args.learning_rate, validate=True)

05/23/2022 08:20:54 - INFO - root -   ***** Running training *****
05/23/2022 08:20:54 - INFO - root -     Num examples = 100
05/23/2022 08:20:54 - INFO - root -     Num Epochs = 8
05/23/2022 08:20:54 - INFO - root -     Total train batch size (w. parallel, distributed & accumulation) = 8
05/23/2022 08:20:54 - INFO - root -     Gradient Accumulation steps = 1
05/23/2022 08:20:54 - INFO - root -     Total optimization steps = 104


/usr/local/lib/python3.7/dist-packages/pytorch_lamb/lamb.py:96: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


05/23/2022 08:20:59 - INFO - root -   Running evaluation
05/23/2022 08:20:59 - INFO - root -     Num examples = 50
05/23/2022 08:20:59 - INFO - root -     Batch size = 16


05/23/2022 08:21:00 - INFO - root -   eval_loss after epoch 1: 0.5728354081511497: 
05/23/2022 08:21:00 - INFO - root -   eval_accuracy after epoch 1: 0.76: 
05/23/2022 08:21:00 - INFO - root -   lr after epoch 1: 0.004809698831278217
05/23/2022 08:21:00 - INFO - root -   train_loss after epoch 1: 0.7007812995177048
05/23/2022 08:21:00 - INFO - root -   



/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

05/23/2022 08:21:24 - INFO - root -   eval_loss after epoch 5: 0.8021617252379656: 
05/23/2022 08:21:24 - INFO - root -   eval_accuracy after epoch 5: 0.86: 
05/23/2022 08:21:24 - INFO - root -   lr after epoch 5: 0.0015432914190872758
05/23/2022 08:21:24 - INFO - root -   train_loss after epoch 5: 0.2086436680983752
05/23/2022 08:21:24 - INFO - root -   

05/23/2022 08:21:29 - INFO - root -   Running evaluation
05/23/2022 08:21:29 - INFO - root -     Num examples = 50
05/23/2022 08:21:29 - INFO - root -     Batch size = 16


05/23/2022 08:21:30 - INFO - root -   eval_loss after epoch 6: 0.9326862530251674: 
05/23/2022 08:21:30 - INFO - root -   eval_accuracy after epoch 6: 0.86: 
05/23/2022 08:21:30 - INFO - root -   lr after epoch 6: 0.0007322330470336313
05/23/2022 08:21:30 - INFO - root -   train_loss after epoch 6: 0.052237032893642135
05/23/2022 08:21:30 - INFO - root -   

05/23/2022 08:21:35 - INFO - root -   Running evaluation
05/23/2022 08:21:35 - INFO - root -     Num examples = 50
05/23/2022 08:21:35 - INFO - root -     Batch size = 16


05/23/2022 08:21:36 - INFO - root -   eval_loss after epoch 7: 0.8644221118702262: 
05/23/2022 08:21:36 - INFO - root -   eval_accuracy after epoch 7: 0.86: 
05/23/2022 08:21:36 - INFO - root -   lr after epoch 7: 0.00019030116872178316
05/23/2022 08:21:36 - INFO - root -   train_loss after epoch 7: 0.11741307552718074
05/23/2022 08:21:36 - INFO - root -   

05/23/2022 08:21:42 - INFO - root -   Running evaluation
05/23/2022 08:21:42 - INFO - root -     Num examples = 50
05/23/2022 08:21:42 - INFO - root -     Batch size = 16


05/23/2022 08:21:43 - INFO - root -   eval_loss after epoch 8: 0.7857903833464661: 
05/23/2022 08:21:43 - INFO - root -   eval_accuracy after epoch 8: 0.88: 
05/23/2022 08:21:43 - INFO - root -   lr after epoch 8: 0.0
05/23/2022 08:21:43 - INFO - root -   train_loss after epoch 8: 0.00013590134067971332
05/23/2022 08:21:43 - INFO - root -   



(104, 0.2724351586762168)

In [15]:
learner.save_model()

In [16]:
from fast_bert.prediction import BertClassificationPredictor

In [17]:
predictor = BertClassificationPredictor(OUT_PATH/'model_out', LABEL_PATH, multi_label=False, model_type=args.model_type)
predictor

In [18]:
predictor.predict_batch(["i hate you"])

05/23/2022 08:22:00 - INFO - root -   Writing example 0 of 1


[[('0', 0.9999613761901855), ('1', 3.858158379443921e-05)]]